In [1]:
from mmdet.datasets.builder import DATASETS
from mmdet.datasets import CocoDataset
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmcv import Config

In [2]:
# 데이터 셋 등록
from mmdet.datasets import DATASETS
from mmdet.datasets import CocoDataset

@DATASETS.register_module(force = True)
class CapsuleDataset(CocoDataset):
    CLASSES = ("01_ulcer", "02_mass", "04_lymph", "05_bleeding")

In [5]:
#!wget -O /home/bgw2001/mmdetection/checkpoints/centernet_resnet18_dcnv2_140e_coco_20210702_155131-c8cd631f.pth https://download.openmmlab.com/mmdetection/v2.0/centernet/centernet_resnet18_dcnv2_140e_coco/centernet_resnet18_dcnv2_140e_coco_20210702_155131-c8cd631f.pth

--2021-12-04 21:13:48--  https://download.openmmlab.com/mmdetection/v2.0/centernet/centernet_resnet18_dcnv2_140e_coco/centernet_resnet18_dcnv2_140e_coco_20210702_155131-c8cd631f.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.75.20.5
Connecting to download.openmmlab.com (download.openmmlab.com)|47.75.20.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57832952 (55M) [application/octet-stream]
Saving to: ‘/home/bgw2001/mmdetection/checkpoints/centernet_resnet18_dcnv2_140e_coco_20210702_155131-c8cd631f.pth’

/home/bgw2001/mmdet 100%[===================>]  55.15M  9.79MB/s    in 5.6s    

2021-12-04 21:13:55 (9.86 MB/s) - ‘/home/bgw2001/mmdetection/checkpoints/centernet_resnet18_dcnv2_140e_coco_20210702_155131-c8cd631f.pth’ saved [57832952/57832952]



In [3]:
from mmcv import Config

config_file = "/home/bgw2001/mmdetection/configs/centernet/centernet_resnet18_dcnv2_140e_coco.py"
checkpoints_file = "/home/bgw2001/mmdetection/checkpoints/centernet_resnet18_dcnv2_140e_coco_20210702_155131-c8cd631f.pth"

cfg = Config.fromfile(config_file)
print(cfg.pretty_text)

dataset_type = 'CocoDataset'
data_root = 'data/coco/'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile', to_float32=True, color_type='color'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='PhotoMetricDistortion',
        brightness_delta=32,
        contrast_range=(0.5, 1.5),
        saturation_range=(0.5, 1.5),
        hue_delta=18),
    dict(
        type='RandomCenterCropPad',
        crop_size=(512, 512),
        ratios=(0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3),
        mean=[0, 0, 0],
        std=[1, 1, 1],
        to_rgb=True,
        test_pad_mode=None),
    dict(type='Resize', img_scale=(512, 512), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='DefaultFormatBundle'),
    dict(type='Collec

In [4]:
from mmdet.apis import set_random_seed

cfg.seed = 0
set_random_seed(0, deterministic = False)

cfg.dataset_type = "CapsuleDataset"
cfg.data_root = "/media/bgw2001/One Touch/project/HardCover_ObjectDetection/data/images/"

cfg.workers_per_gpu = 2
cfg.samples_per_gpu = 16 * 8

cfg.data.train.dataset.type = "CapsuleDataset"
cfg.data.train.dataset.data_root = "/media/bgw2001/One Touch/project/HardCover_ObjectDetection/data/images/"
cfg.data.train.dataset.img_prefix = "train"
cfg.data.train.dataset.ann_file = "train.json"

cfg.data.val.type = "CapsuleDataset"
cfg.data.val.data_root = "/media/bgw2001/One Touch/project/HardCover_ObjectDetection/data/images/"
cfg.data.val.img_prefix = "valid"
cfg.data.val.ann_file = "valid.json"

cfg.data.test.type = "CapsuleDataset"
cfg.data.test.data_root = "/media/bgw2001/One Touch/project/HardCover_ObjectDetection/data/images/"
cfg.data.test.img_prefix = "test"
cfg.data.test.ann_file = "test.json"

cfg.optimizer.lr = 0.02 

cfg.lr_config.policy = 'step'
#cfg.lr_config.warmup = 'None'

cfg.runner.max_epochs = 28
cfg.log_config.interval = 30

cfg.work_dir = "/home/bgw2001/mmdetection/work_dir/CenterNet/"
cfg.load_from = checkpoints_file
cfg.gpu_ids = range(1)

cfg.model.bbox_head.num_classes = 4

albu_train_transforms = [
    dict(
        type='HorizontalFlip',
        p=0.5)
]

train_pipeline = [
    dict(type='LoadImageFromFile', to_float32=True, color_type='color'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='PhotoMetricDistortion',
        brightness_delta=32,
        contrast_range=(0.5, 1.5),
        saturation_range=(0.5, 1.5),
        hue_delta=18),
    dict(
        type='RandomCenterCropPad',
        crop_size=(512, 512),
        ratios=(0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3),
        mean=[0, 0, 0],
        std=[1, 1, 1],
        to_rgb=True,
        test_pad_mode=None),
    dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
        type='BboxParams',
        format='pascal_voc',
        label_fields=['gt_labels'],
        min_visibility=0.0,
        filter_lost_elements=True),
        keymap=dict(img='image', gt_bboxes='bboxes'),
        update_pad_shape=False,
        skip_img_without_anno=True),
    dict(type='Resize', img_scale=(512, 512), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]

cfg.train_pipeline = train_pipeline
cfg.data.train.dataset.pipeline = train_pipeline

In [5]:
cfg.dump("/home/bgw2001/mmdetection/work_dir/CenterNet/centernet_conf.py")

In [5]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.38s)
creating index...
index created!


In [6]:
datasets[0]

In [7]:
model = build_detector(cfg.model, train_cfg = cfg.get("train_cfg"), test_cfg = cfg.get("test_cfg"))
model.CLASSES = datasets[0].CLASSES
print(model.CLASSES)

('01_ulcer', '02_mass', '04_lymph', '05_bleeding')


In [8]:
train_detector(model, datasets, cfg, distributed=False, validate=True)

2021-12-04 21:20:57,739 - mmdet - INFO - load checkpoint from /home/bgw2001/mmdetection/checkpoints/centernet_resnet18_dcnv2_140e_coco_20210702_155131-c8cd631f.pth
2021-12-04 21:20:57,739 - mmdet - INFO - Use load_from_local loader
2021-12-04 21:20:57,780 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.heatmap_head.2.weight: copying a param with shape torch.Size([80, 64, 1, 1]) from checkpoint, the shape in current model is torch.Size([4, 64, 1, 1]).
size mismatch for bbox_head.heatmap_head.2.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([4]).
2021-12-04 21:20:57,781 - mmdet - INFO - Start running, host: bgw2001@bgw2001-All-Series, work_dir: /home/bgw2001/mmdetection/work_dir/CenterNet
2021-12-04 21:20:57,781 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook         

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


/home/bgw2001/anaconda3/envs/py3.8/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
2021-12-04 21:21:11,312 - mmdet - INFO - Epoch [1][30/11742]	lr: 5.994e-04, eta: 1 day, 17:10:44, time: 0.451, data_time: 0.111, memory: 2625, loss_center_heatmap: 11606.2069, loss_wh: 1.6361, loss_offset: 0.2568, loss: 11608.0999, grad_norm: 79972.6956
2021-12-04 21:21:21,464 - mmdet - INFO - Epoch [1][60/11742]	lr: 1.199e-03, eta: 1 day, 12:02:13, time: 0.338, data_time: 0.012, memory: 2625, loss_center_heatmap: 4.9365, loss_wh: 1.2107, loss_offset: 0.2519, loss: 6.3991, grad_norm: 45.9404
2021-12-04 21:21:32,078 - mmdet - INFO - Epoch [1][90/11742]	lr: 1.798e-03,

2021-12-04 21:26:45,107 - mmdet - INFO - Epoch [1][990/11742]	lr: 1.978e-02, eta: 1 day, 7:56:35, time: 0.342, data_time: 0.014, memory: 2625, loss_center_heatmap: 3.3133, loss_wh: 0.9230, loss_offset: 0.2533, loss: 4.4896, grad_norm: 5.2745
2021-12-04 21:26:55,409 - mmdet - INFO - Epoch [1][1020/11742]	lr: 2.000e-02, eta: 1 day, 7:55:13, time: 0.343, data_time: 0.014, memory: 2625, loss_center_heatmap: 3.2427, loss_wh: 0.9451, loss_offset: 0.2590, loss: 4.4468, grad_norm: 5.0182
2021-12-04 21:27:05,720 - mmdet - INFO - Epoch [1][1050/11742]	lr: 2.000e-02, eta: 1 day, 7:53:58, time: 0.344, data_time: 0.014, memory: 2625, loss_center_heatmap: 3.3166, loss_wh: 0.9747, loss_offset: 0.2527, loss: 4.5440, grad_norm: 6.4875
2021-12-04 21:27:19,650 - mmdet - INFO - Epoch [1][1080/11742]	lr: 2.000e-02, eta: 1 day, 8:11:04, time: 0.464, data_time: 0.136, memory: 2625, loss_center_heatmap: 3.6359, loss_wh: 1.0660, loss_offset: 0.2439, loss: 4.9458, grad_norm: 7.2609
2021-12-04 21:27:29,939 - mmd

2021-12-04 21:32:49,611 - mmdet - INFO - Epoch [1][2010/11742]	lr: 2.000e-02, eta: 1 day, 8:08:40, time: 0.341, data_time: 0.013, memory: 2625, loss_center_heatmap: 3.2338, loss_wh: 0.9422, loss_offset: 0.2524, loss: 4.4284, grad_norm: 5.4297
2021-12-04 21:32:59,842 - mmdet - INFO - Epoch [1][2040/11742]	lr: 2.000e-02, eta: 1 day, 8:07:26, time: 0.341, data_time: 0.014, memory: 2625, loss_center_heatmap: 3.1576, loss_wh: 0.8063, loss_offset: 0.2505, loss: 4.2144, grad_norm: 4.6684
2021-12-04 21:33:10,337 - mmdet - INFO - Epoch [1][2070/11742]	lr: 2.000e-02, eta: 1 day, 8:06:56, time: 0.350, data_time: 0.020, memory: 2625, loss_center_heatmap: 3.0587, loss_wh: 0.8338, loss_offset: 0.2499, loss: 4.1424, grad_norm: 4.3658
2021-12-04 21:33:20,619 - mmdet - INFO - Epoch [1][2100/11742]	lr: 2.000e-02, eta: 1 day, 8:05:53, time: 0.343, data_time: 0.015, memory: 2625, loss_center_heatmap: 2.9804, loss_wh: 0.8432, loss_offset: 0.2520, loss: 4.0756, grad_norm: 4.7513
2021-12-04 21:33:30,855 - mm

...

2021-12-06 06:35:24,898 - mmdet - INFO - Epoch [28][11550/11742]	lr: 2.000e-04, eta: 0:01:05, time: 0.342, data_time: 0.013, memory: 2625, loss_center_heatmap: 0.9434, loss_wh: 0.2229, loss_offset: 0.2339, loss: 1.4002, grad_norm: 7.1059
2021-12-06 06:35:35,120 - mmdet - INFO - Epoch [28][11580/11742]	lr: 2.000e-04, eta: 0:00:55, time: 0.341, data_time: 0.013, memory: 2625, loss_center_heatmap: 1.0205, loss_wh: 0.2469, loss_offset: 0.2345, loss: 1.5019, grad_norm: 7.4000
2021-12-06 06:35:45,316 - mmdet - INFO - Epoch [28][11610/11742]	lr: 2.000e-04, eta: 0:00:45, time: 0.340, data_time: 0.013, memory: 2625, loss_center_heatmap: 0.9783, loss_wh: 0.2497, loss_offset: 0.2365, loss: 1.4645, grad_norm: 7.2773
2021-12-06 06:35:55,558 - mmdet - INFO - Epoch [28][11640/11742]	lr: 2.000e-04, eta: 0:00:34, time: 0.341, data_time: 0.013, memory: 2625, loss_center_heatmap: 0.9225, loss_wh: 0.2374, loss_offset: 0.2299, loss: 1.3898, grad_norm: 6.8235
2021-12-06 06:36:05,818 - mmdet - INFO - Epoch [

[>>>>>>>>>>>>>>>>>>>>>>>>] 12524/12524, 65.3 task/s, elapsed: 192s, ETA:     0s

2021-12-06 06:39:55,898 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=2.97s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=30.23s).
Accumulating evaluation results...
DONE (t=28.76s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.456
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.813
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.457
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.130
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.424
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.463
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.570
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.570
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.570
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | ma

2021-12-06 06:41:03,759 - mmdet - INFO - Epoch(val) [28][12524]	bbox_mAP: 0.4560, bbox_mAP_50: 0.8130, bbox_mAP_75: 0.4570, bbox_mAP_s: 0.1300, bbox_mAP_m: 0.4240, bbox_mAP_l: 0.4630, bbox_mAP_copypaste: 0.456 0.813 0.457 0.130 0.424 0.463


In [7]:
import json
import json
import base64
import numpy as np
import cv2
import pandas as pd
import os

from PIL import Image
from io import BytesIO
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
from mmdet.apis import multi_gpu_test, single_gpu_test
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmdet.datasets import (build_dataloader, build_dataset, replace_ImageToTensor)
from mmdet.datasets.builder import DATASETS
from mmdet.datasets import CocoDataset
from tqdm import tqdm
from glob import glob
from mmcv import Config

In [9]:
# 데이터 셋 등록
@DATASETS.register_module(force = True)
class CapsuleDataset(CocoDataset):
    CLASSES = ("01_ulcer", "02_mass", "04_lymph", "05_bleeding")

config_file = "/home/bgw2001/mmdetection/work_dir/CenterNet/centernet_conf.py"
checkpoint_file = "/home/bgw2001/mmdetection/work_dir/CenterNet/epoch_28.pth"

In [10]:
def submission(model, paths, score_thr = 0.5):
    file_names = []
    class_ids = []
    confidences = []
    
    point1_x = []
    point1_y = []
    
    point2_x = []
    point2_y = []

    point3_x = []
    point3_y = []

    point4_x = []
    point4_y = []
    
    for path in tqdm(paths):
        
        with open(path, "r") as f:
            data = json.load(f)
        
        img = Image.open(BytesIO(base64.b64decode(data['imageData'])))
        img = np.array(img, np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        results = inference_detector(model, img)
        
        index = path.rfind("/")
        file_name = path[index + 1:]
        
        for ind,result in enumerate(results):
            for i in range(len(result)):
                if result[i,4] > score_thr:
                    file_names.append(file_name)
                    xmin = result[i,0]
                    ymin = result[i,1]
                    xmax = result[i,2]
                    ymax = result[i,3]
            
                    point1_x.append(xmin)
                    point1_y.append(ymin)
            
                    point2_x.append(xmax)
                    point2_y.append(ymin)
            
                    point3_x.append(xmax)
                    point3_y.append(ymax)
            
                    point4_x.append(xmin)
                    point4_y.append(ymax)
            
                    class_ids.append(ind)
                    confidences.append(result[i,4])
    
    submission = pd.DataFrame({"file_name":file_names,
                               "class_id":class_ids,
                               "confidence":confidences,
                               "point1_x":point1_x,
                               "point1_y":point1_y,
                               "point2_x":point2_x,
                               "point2_y":point2_y,
                               "point3_x":point3_x,
                               "point3_y":point3_y,
                               "point4_x":point4_x,
                               "point4_y":point4_y})
    return submission

In [11]:
os.chdir("/media/bgw2001/One Touch/project/HardCover_ObjectDetection/")
!ls -ail

total 2560
700 drwxr-xr-x 10 bgw2001 bgw2001 262144 12월  4 16:23 .
683 drwxr-xr-x 18 bgw2001 bgw2001 262144 11월 17 17:26 ..
711 drwxr-xr-x  6 bgw2001 bgw2001 262144 12월  4 21:08 code
709 drwxr-xr-x  7 bgw2001 bgw2001 262144 12월  4 11:35 data
715 drwxr-xr-x  3 bgw2001 bgw2001 262144 12월  4 17:22 emsanble
717 drwxr-xr-x  2 bgw2001 bgw2001 262144 11월 25 11:40 .ipynb_checkpoints
713 drwxr-xr-x  3 bgw2001 bgw2001 262144 11월 28 18:52 practice_code
710 drwxr-xr-x  3 bgw2001 bgw2001 262144 11월 19 14:50 ref
712 drwxr-xr-x  2 bgw2001 bgw2001 262144 12월  4 15:53 submission
714 drwxr-xr-x  4 bgw2001 bgw2001 262144 12월  4 11:42 work_dir


In [12]:
model = init_detector(config_file, checkpoint_file)

paths = glob("./data/test/*.json")

Use load_from_local loader


In [13]:
print(len(paths))

20874


In [16]:
df = submission(model, paths, score_thr = 0.4)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 20874/20874 [08:08<00:00, 42.70it/s]


In [17]:
print("데이터 프레임 모양:",df.shape)
df.head()

데이터 프레임 모양: (19286, 11)


,file_name,class_id,confidence,point1_x,point1_y,point2_x,point2_y,point3_x,point3_y,point4_x,point4_y
0,test_200001.json,0,0.580843,120.246628,327.820068,267.749390,327.820068,267.749390,468.471802,120.246628,468.471802
1,test_200001.json,0,0.492727,406.198853,189.421280,525.806763,189.421280,525.806763,310.616028,406.198853,310.616028
2,test_200002.json,1,0.693104,221.590546,346.322540,341.939240,346.322540,341.939240,473.830048,221.590546,473.830048
3,test_200003.json,1,0.818653,130.928268,7.429314,304.764648,7.429314,304.764648,189.059113,130.928268,189.059113
4,test_200003.json,1,0.791847,227.721893,136.640396,320.583466,136.640396,320.583466,258.696991,227.721893,258.696991


In [18]:
df['class_id'] = df['class_id'] + 1
df.head()

,file_name,class_id,confidence,point1_x,point1_y,point2_x,point2_y,point3_x,point3_y,point4_x,point4_y
0,test_200001.json,1,0.580843,120.246628,327.820068,267.749390,327.820068,267.749390,468.471802,120.246628,468.471802
1,test_200001.json,1,0.492727,406.198853,189.421280,525.806763,189.421280,525.806763,310.616028,406.198853,310.616028
2,test_200002.json,2,0.693104,221.590546,346.322540,341.939240,346.322540,341.939240,473.830048,221.590546,473.830048
3,test_200003.json,2,0.818653,130.928268,7.429314,304.764648,7.429314,304.764648,189.059113,130.928268,189.059113
4,test_200003.json,2,0.791847,227.721893,136.640396,320.583466,136.640396,320.583466,258.696991,227.721893,258.696991


In [20]:
df.to_csv("./submission/centernet_summisson.csv",index=False)